# 1. Import Packages

In [ ]:
#------Basics-------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#Import train_test_split
from sklearn.model_selection import train_test_split
from tqdm import tqdm

#------CSV Download-------
# install the gdown package in Colab
!pip install gdown

# downloading the .csv file from Google Drive using gdown
import gdown

#------Optimization-------
!pip install cvxpy

import cvxpy as cp

#------Models----------
import pickle


# 2. Load Datasets

## Collision Dataset (named df)

In [ ]:
# set viewing permission to 'Anyone with the link' if not already done
# file ID from the Google Drive link -> https://drive.google.com/file/d/FILE_ID/view?usp=sharing
file_id = '14KpYro56AbayWBQZ6CJfootZ-lC_SseK'
url = f'https://drive.google.com/uc?id={file_id}'

# download the file and save the name of the file
final = 'final_dataset.csv'
gdown.download(url, final, quiet=False)

# loading the .csv into a DataFrame
import pandas as pd

df = pd.read_csv(final)
df.head()

Downloading...
From: https://drive.google.com/uc?id=14KpYro56AbayWBQZ6CJfootZ-lC_SseK
To: /content/final_dataset.csv
100%|██████████| 21.2M/21.2M [00:00<00:00, 144MB/s]


,OCC_DATE,OCC_MONTH,OCC_DOW,OCC_YEAR,OCC_HOUR,COLLISION_LONGITUDE,COLLISION_LATITUDE,INTERSECTION_ID,INTERSECTION_CLASSIFICATION,RED_LIGHT_CAMERA,Temp (°C),Rel Hum (%),Precip. Amount (mm),Visibility (km),Stn Press (kPa),SPD_KM,NUM_TRAFFIC_SIGNALS
0,2020-01-01,January,Wednesday,2020,6.0,-79.197483,43.758945,13448608,MNRSL,0,-0.6,73.0,0.0,16.1,98.99,40,1.0
1,2020-01-01,January,Wednesday,2020,1.0,-79.550892,43.677196,13462655,MJRSL,0,-0.1,70.0,0.0,16.1,98.84,60,1.0
2,2020-01-01,January,Wednesday,2020,5.0,-79.336669,43.797327,13443799,MNRSL,0,-0.5,72.0,0.2,16.1,98.96,40,0.0
3,2020-01-01,January,Wednesday,2020,2.0,-79.393760,43.642772,13467856,MNRSL,0,-0.1,67.0,0.0,16.1,98.87,60,1.0
4,2020-01-01,January,Wednesday,2020,4.0,-79.312104,43.748833,13450356,MNRSL,0,-0.7,75.0,0.0,16.1,98.96,60,0.0


## Ambulance & Candidate Locations (called current_df, candidate_df)

In [ ]:
# set viewing permission to 'Anyone with the link' if not already done
# file ID from the Google Drive link -> https://drive.google.com/file/d/FILE_ID/view?usp=sharing
file_id = '1RLkgm8987SJIiLGVk88iuv200VnQuW_f'
url = f'https://drive.google.com/uc?id={file_id}'

# download the fire file and save the name of the file
candidate_locations = 'candidate_df.csv'
gdown.download(url, candidate_locations, quiet=False)

# file ID from the Google Drive link -> https://drive.google.com/file/d/FILE_ID/view?usp=sharing
file_id = '1UI3NlAUUplScXkhYwYLihfhQrB5BWkrI'
url = f'https://drive.google.com/uc?id={file_id}'
# download the ambulance file and save the name of the file
current_locations = 'current_locations.csv'
gdown.download(url, current_locations, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1RLkgm8987SJIiLGVk88iuv200VnQuW_f
To: /content/candidate_df.csv
100%|██████████| 2.69k/2.69k [00:00<00:00, 7.20MB/s]
Downloading...
From: https://drive.google.com/uc?id=1UI3NlAUUplScXkhYwYLihfhQrB5BWkrI
To: /content/current_locations.csv
100%|██████████| 1.62k/1.62k [00:00<00:00, 4.41MB/s]


'current_locations.csv'

In [ ]:
#Set df for candidates
candidate_df = pd.read_csv(candidate_locations)

#Set df for current locations
current_df = pd.read_csv(current_locations)

In [ ]:
#Print df size
print("Candidate df:")
print(candidate_df.shape)
print("Current df:")
print(current_df.shape)

#Print df head
candidate_df.head()

Candidate df:
(72, 2)
Current df:
(46, 2)


,LONGITUDE,LATITUDE
0,-79.163602,43.794211
1,-79.438867,43.645086
2,-79.406781,43.748540
3,-79.617638,43.738108
4,-79.428572,43.719798


In [ ]:
#Print df head
current_df.head()

,LONGITUDE,LATITUDE
0,-79.242870,43.823993
1,-79.444286,43.776286
2,-79.466532,43.761791
3,-79.415355,43.772636
4,-79.226258,43.742000


# 3. Prediction Models

## Import Models


In [ ]:
# Download KDE model
file_id = '1oMeHIK8u6-d-XOaI7Pj0rISQS8t2NtU1'
url = f'https://drive.google.com/uc?id={file_id}'
# Download the KDE file and save the name of the file
gdown.download(url, 'KDE.sav', quiet=False)

# Download Logistic model
file_id = '1ZiBSGwrIkEkrwy7M4lZFyiORwofwBgrh'
url = f'https://drive.google.com/uc?id={file_id}'
# Download the Logistic file and save the name of the file
gdown.download(url, 'LogisticRegression.sav', quiet=False)

# Download RF model
file_id = '1JaZ6f_JYtjz40yDoW-nHo57XAAp8Qstv'
url = f'https://drive.google.com/uc?id={file_id}'
# Download the RF file and save the name of the file
gdown.download(url, 'RandomForestClassifier.sav', quiet=False)

# KDE model
kde_model = pickle.load(open('KDE.sav', 'rb'))

# Logistic model
logistic_model = pickle.load(open('LogisticRegression.sav', 'rb'))

# Random Forest model
rf_model = pickle.load(open('RandomForestClassifier.sav', 'rb'))

Downloading...
From: https://drive.google.com/uc?id=1oMeHIK8u6-d-XOaI7Pj0rISQS8t2NtU1
To: /content/KDE.sav
100%|██████████| 3.79M/3.79M [00:00<00:00, 60.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZiBSGwrIkEkrwy7M4lZFyiORwofwBgrh
To: /content/LogisticRegression.sav
100%|██████████| 3.49k/3.49k [00:00<00:00, 8.03MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1JaZ6f_JYtjz40yDoW-nHo57XAAp8Qstv
From (redirected): https://drive.google.com/uc?id=1JaZ6f_JYtjz40yDoW-nHo57XAAp8Qstv&confirm=t&uuid=ce92ec78-0cc2-41c3-b3ce-e18835c759d0
To: /content/RandomForestClassifier.sav
100%|██████████| 2.22G/2.22G [00:26<00:00, 82.8MB/s]


## KDE Risk Function

In [ ]:
def kde_risk(model, traffic_data):
    """
    Calculates the KDE scores for a given dataset.

    Parameters:
    - model: A trained Kernel Density Estimation (KDE) model.
    - traffic_data: The dataset for which to calculate the KDE scores.

    Returns:
    - kde_scores_normalized: The normalized KDE scores (between 0 and 1).
    """

    # Get the KDE scores
    kde_scores_raw = model.score_samples(traffic_data)

    # Apply exponential function and normalize
    kde_normalized = np.exp(kde_scores_raw) / (np.exp(kde_scores_raw)).sum()

    # Return raw and normalized KDE scores
    return kde_normalized

## Logistic Risk Function
Takes in locations and returns an array of risks

In [ ]:
# Prediction function with tqdm progress bar
def logistic_risk(model, traffic_data):
    """
    Predicts the collision risk for each row in the traffic_data using a logistic regression model.

    Parameters:
    - model: A trained logistic regression model
    - traffic_data: DataFrame containing the features for each collision instance.

    Returns:
    - risk: A list or array of the predicted risk for each row in traffic_data.
    """

    #Set risk array - returns array for each row with probability of class 1 and class 2
    risk_probabilities = model.predict_proba(traffic_data)

    #Get risk of collision class (2nd column)
    risk = risk_probabilities[:, 1]

    #Return risk
    return risk



## Random Forest Risk Function
Takes in locations and returns an array of risks

In [ ]:
# Prediction function with tqdm progress bar for random forest model
def rf_risk(model, traffic_data):
    """
    Predicts the collision risk for each row in the traffic_data using a random forest model.

    Parameters:
    - model: A trained random forest model
    - traffic_data: DataFrame containing the features for each collision instance.

    Returns:
    - risk: A list or array of the predicted risk for each row in traffic_data.
    """

    # Set risk array - returns array for each row with probability of class 1 and class 2
    risk_probabilities = model.predict_proba(traffic_data)  # Make sure traffic_data is passed correctly

    # Get risk of collision class (2nd column, corresponding to class 1 probability)
    risk = risk_probabilities[:, 1]

    # Return risk
    return risk


# 4. Gurobi Model

## Set up model

In [ ]:
!pip install gurobipy

In [ ]:
from google.colab import files
uploaded = files.upload()

# Move the uploaded license to the home directory
import shutil
shutil.move(list(uploaded.keys())[0], '/root/gurobi.lic')

from gurobipy import Model
print("Gurobi license successfully recognized!")

Saving gurobi.lic to gurobi.lic
Gurobi license successfully recognized!


## Final Model

In [ ]:
#Import gurobipy
import gurobipy as gp
from gurobipy import GRB

#Define optimization function
def optimize_ambulance_placement(risk, d_existing, d_candidate, num_new_ambulances, candidate_coordinates):
    """
    Optimizes the placement of new ambulances.

    Parameters:
    - risk (list[float]): Array of risk values for demand points (size n).
    - d_existing (list[list[float]]): Distance matrix (n x p) from demand points to p existing ambulances.
    - d_candidate (list[list[float]]): Distance matrix (n x m) from demand points to m candidate ambulances.
    - num_new_ambulances (int): Number of new ambulances to place.
    - df_coordinates (pd.DataFrame): DataFrame with coordinates for existing and candidate locations.

    Returns:
    - pd.DataFrame: Coordinates of selected candidate ambulances.
    """
    #---------Get dimensions from inputs---------
    # Number of demand points
    n = risk.size
    # Number of existing ambulances
    p = d_existing.shape[1]
    # Number of candidate ambulances
    m = d_candidate.shape[1]

    #---------Initialize Gurobi model-----------
    model = gp.Model("AmbulancePlacement")

    #-------Define decision variables----------
    # Candidate ambulance placement
    y = model.addVars(m, vtype=GRB.BINARY, name="y")
    # Demand assignment to existing ambulances
    z_existing = model.addVars(n, p, vtype=GRB.CONTINUOUS, name="z_existing")
    # Demand assignment to candidate ambulances
    z_candidate = model.addVars(n, m, vtype=GRB.CONTINUOUS, name="z_candidate")

    #---------- Define the objective function-----
    # Aggregate the total weighted distance for all demand points
    objective = gp.quicksum(
        # Scale the distance by the risk factor for demand point j
        risk[j] * (
            # Contribution from existing ambulances
            gp.quicksum(d_existing.iloc[j, k] * z_existing[j, k] for k in range(p))  +
            # Contribution from candidate ambulances
            gp.quicksum(d_candidate.iloc[j, i] * z_candidate[j, i] for i in range(m))
        )
        # Iterate over all demand points
        for j in range(n)
    )

    # Set the model's objective to minimize the total weighted distance
    model.setObjective(objective, GRB.MINIMIZE)

    #-------Add constraints----------
    # Each demand point must be assigned to exactly one ambulance
    model.addConstrs(
        (gp.quicksum(z_existing[j, k] for k in range(p)) +
         gp.quicksum(z_candidate[j, i] for i in range(m)) == 1
         for j in range(n)),
        name="MaxOneAmbulanceAssigned"
    )

    # Exactly `num_new_ambulances` ambulances can be placed
    model.addConstr(
        gp.quicksum(y[i] for i in range(m)) == num_new_ambulances,
        name="AmbulanceQuantityLimit"
    )

    # Ambulance i can only be assigned to demand point j if we have an ambulance at i
    model.addConstrs(
        (z_candidate[j, i] <= y[i] for j in range(n) for i in range(m)),
        name="AssignmentRestriction"
    )

    # >= zero constraint for z
    model.addConstrs(z_candidate[j, i] >= 0 for j in range(n) for i in range(m))

    #-------Solve the model------
    model.optimize()

    #-----------Extract results--------
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found.")

        # Print the objective function value
        objective_value = model.objVal
        print(f"Optimal Weighted Distance Value: {objective_value}")

        # Calculate the average objective function value
        average_objective_value = objective_value / n
        print(f"Average Weighted Distance Value (over all demand locations): {average_objective_value}")

        # Get selected candidate locations (index for y = 1)
        selected_locations = [i for i in range(m) if y[i].x > 0.5]
        print("Selected candidate locations (indices):", selected_locations)

        # Filter the candidate coordinates to the selected coordinates
        selected_coordinates = candidate_coordinates.iloc[selected_locations][["LONGITUDE", "LATITUDE"]]

        return selected_coordinates
    else:
        print("No optimal solution found.")
        return None



## Import Distance CSV (Train)

In [ ]:
#------------All distance dataset-----------

# Set viewing permission to 'Anyone with the link' if not already done
# File ID from the Google Drive link -> https://drive.google.com/file/d/FILE_ID/view?usp=sharing
file_id = '1qJgwruuDtEN5Bddf-05T2IYUKSGs1xJ7'
url = f'https://drive.google.com/uc?id={file_id}'

# Download the file and save the name of the file
all = 'all.csv'
gdown.download(url, all, quiet=True)

# Loading the .csv into a DataFrame
d_all = pd.read_csv(all)

In [ ]:
#-------------------Current distance dataset-----------

# Set viewing permission to 'Anyone with the link' if not already done
# File ID from the Google Drive link -> https://drive.google.com/file/d/FILE_ID/view?usp=sharing
file_id = '1dUrS7VrkMSr527a-TA-PTSu2LHyqaQSR'
url = f'https://drive.google.com/uc?id={file_id}'

# Download the file and save the name of the file
current = 'current.csv'
gdown.download(url, current, quiet=True)

# Loading the .csv into a DataFrame
d_current = pd.read_csv(current)

In [ ]:
#------------Candidate distance dataset-------------
# Set viewing permission to 'Anyone with the link' if not already done
# File ID from the Google Drive link -> https://drive.google.com/file/d/FILE_ID/view?usp=sharing
file_id = '12pSYl4eR5c-PgDjMQ6xGJNvpEsXcCHgN'
url = f'https://drive.google.com/uc?id={file_id}'

# Download the file and save the name of the file
candidate = 'candidate.csv'
gdown.download(url, candidate, quiet=True)

# Loading the .csv into a DataFrame
d_candidate = pd.read_csv(candidate)

## Import Hot-encoded Train DF

In [ ]:
#------------Hotencoded X_train for 10k points-------------
# Set viewing permission to 'Anyone with the link' if not already done
# File ID from the Google Drive link -> https://drive.google.com/file/d/FILE_ID/view?usp=sharing
file_id = '1NS_1PkiwLcKRO6CR_08XqIyHeGuG3V09'
url = f'https://drive.google.com/uc?id={file_id}'

# Download the file and save the name of the file
train_dataset_encoded = 'train_dataset_encoded.csv'
gdown.download(url, train_dataset_encoded, quiet=True)

# Loading the .csv into a DataFrame
train_encoded_df = pd.read_csv(train_dataset_encoded)

In [ ]:
# See data
train_encoded_df.head()

,OCC_MONTH_1,OCC_MONTH_10,OCC_MONTH_11,OCC_MONTH_12,OCC_MONTH_2,OCC_MONTH_3,OCC_MONTH_4,OCC_MONTH_5,OCC_MONTH_6,OCC_MONTH_7,...,NUM_TRAFFIC_SIGNALS_1.0,NUM_TRAFFIC_SIGNALS_2.0,NUM_TRAFFIC_SIGNALS_3.0,COLLISION_LONGITUDE,COLLISION_LATITUDE,Temp (°C),Rel Hum (%),Precip. Amount (mm),Visibility (km),Stn Press (kPa)
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,-0.180415,-0.427899,-0.158365,-0.010164,0.128515,0.003220,0.212937
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-0.008389,0.270656,-0.128020,-0.017171,-0.068418,0.202918,0.265933
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.562434,-0.640259,1.264895,-0.323095,-0.186763,0.316248,-0.612049
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2.043411,0.970666,0.928990,-1.442282,-0.186763,0.316248,-0.895893
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.094695,0.295438,0.079583,-0.173568,-0.186763,0.316248,-0.124421


In [ ]:
# Get the list of columns from the training data and the feature names from the logistic model
train_columns = train_encoded_df.columns.tolist()
model_features = logistic_model.feature_names_in_.tolist()

# Compare the lists directly
if train_columns == model_features:
    print("The feature lists are identical in both order and content.")

The feature lists are identical in both order and content.


## Get Risk Arrays

<font color='red'>Get training demand points (sampled 10k locations) </font>

In [ ]:
# Get unique coordinate pairs
matrix = (
    df.drop_duplicates(subset=['INTERSECTION_ID', 'COLLISION_LONGITUDE', 'COLLISION_LATITUDE'])
    .groupby('INTERSECTION_ID', group_keys=False)
    .apply(lambda group: list(zip(group['COLLISION_LONGITUDE'].round(6), group['COLLISION_LATITUDE'].round(6))))
    .rename('COORDINATES')
    .to_frame()
)

# Extracts the tuple from [ ] in some duplicate cases
matrix['COORDINATES'] = matrix['COORDINATES'].apply(
    lambda x: x[0] if isinstance(x, list) else x
)
# Extract the coordinates directly as a numpy array (if needed)
matrix['COORDINATES'] = matrix['COORDINATES'].apply(lambda x: x[0] if isinstance(x, list) else x)

# Randomly sample 10k rows from the DataFrame
matrix = matrix.sample(n=10000, random_state=11)  # random_state is optional for reproducibility

# Get numpy array of coordinates
train_coordinates = np.array(matrix['COORDINATES'].tolist())

<ipython-input-145-543964c22a52>:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: list(zip(group['COLLISION_LONGITUDE'].round(6), group['COLLISION_LATITUDE'].round(6))))


In [ ]:
#Print coordinatees to validate data
train_coordinates

array([[-79.412672,  43.688652],
       [-79.395145,  43.726593],
       [-79.336986,  43.677118],
       ...,
       [-79.429658,  43.794195],
       [-79.489801,  43.612757],
       [-79.252644,  43.727444]])

### KDE

In [ ]:
#Risk for KDE
train_risk_kde = kde_risk(kde_model, train_coordinates)

In [ ]:
#Get risk array statistic
print("KDE risk array statistics:")
print("Min:", train_risk_kde.min())
print("Max:", train_risk_kde.max())
print("Sum:", train_risk_kde.sum())

KDE risk array statistics:
Min: 5.082782176210907e-06
Max: 0.0002948420694884116
Sum: 1.0


### Logistic

In [ ]:
#Risk for Logistic
train_risk_logistic = logistic_risk(logistic_model, train_encoded_df)

In [ ]:
#Print array
train_risk_logistic

array([0.39559429, 0.27477037, 0.21988623, ..., 0.21434834, 0.28335225,
       0.44123783])

### Random Forest

In [ ]:
#Risk for Random Forest
train_risk_rf = rf_risk(rf_model, train_encoded_df)

In [ ]:
train_risk_rf

array([0.36, 0.27, 0.43, ..., 0.7 , 0.3 , 0.39])

## Optimal Solution


In [ ]:
d_current.head()

,"(-79.242870035,43.823992725)","(-79.444286046,43.776285522)","(-79.466531526,43.761790843)","(-79.415354775,43.77263585)","(-79.226258458,43.742000356)","(-79.34068607,43.774403551)","(-79.416483429,43.702609497)","(-79.350519549,43.692888381)","(-79.334662115,43.679542888)","(-79.359074505,43.625400746)",...,"(-79.41318022,43.666047624)","(-79.376672166,43.652429667)","(-79.563826205,43.677147271)","(-79.174053409,43.770826741)","(-79.310711775,43.687089176)","(-79.39365588,43.790127007)","(-79.256237171,43.718005193)","(-79.512384411,43.700598701)","(-79.37787537,43.721937654)","(-79.548588931,43.754779546)"
0,20.309164,10.070026,9.212574,9.341066,16.113651,11.152247,1.581964,5.019375,6.354149,8.249716,...,2.513824,4.960480,12.221927,21.239104,8.200110,11.386457,12.991818,8.125945,4.639226,13.166889
1,16.333174,6.790519,6.942843,5.370925,13.676792,6.884567,3.170683,5.187630,7.142125,11.619985,...,6.886706,8.379277,14.632163,18.427700,8.084323,7.065671,11.203454,9.856126,1.481151,12.719227
2,17.996473,13.997591,14.036770,12.347708,11.456885,10.821745,6.992204,2.063814,0.328069,6.019032,...,6.250587,4.210311,18.242696,16.733214,2.386071,13.365487,7.925412,14.342594,5.970150,19.073194
3,8.022592,20.772227,22.522768,18.430746,4.090057,12.460848,19.752871,15.432388,15.232139,20.809277,...,21.288758,19.754956,31.875296,1.188424,13.212950,16.904096,7.655750,27.165351,16.120387,29.132462
4,11.150439,14.043815,15.235033,11.753257,4.800943,6.953038,11.110563,6.697417,6.809488,12.948756,...,12.542123,11.292361,23.289518,9.948882,5.067726,11.275753,2.400301,18.704208,7.672566,21.563168


In [ ]:
d_candidate.head()

,"(-79.163601604,43.794210767)","(-79.4388667,43.645086441)","(-79.406780749,43.748540077)","(-79.617637808,43.738108289)","(-79.428571867,43.719798152)","(-79.502938169,43.628018538)","(-79.441140591,43.694520303)","(-79.393975841,43.685950806)","(-79.191522029,43.761068166)","(-79.255062704,43.734790143)",...,"(-79.430752157,43.680105152)","(-79.38956419,43.648306534)","(-79.366153969,43.650520808)","(-79.343518186,43.667766682)","(-79.404781725,43.656824596)","(-79.364854202,43.659370534)","(-79.399768302,43.70965676)","(-79.586905852,43.740218706)","(-79.266851071,43.754966946)","(-79.31659701,43.69518746)"
0,23.197639,5.282635,6.676060,17.367235,3.691606,9.908917,2.380113,1.533020,19.510138,13.667216,...,1.736913,4.855982,5.654762,6.026731,3.595490,5.038950,2.555625,15.131805,13.845328,7.758644
1,20.057620,9.721096,2.613321,17.922398,2.790392,13.974661,5.136737,4.520183,16.800527,11.292333,...,5.908771,8.716616,8.774153,7.746851,7.796488,7.861528,1.919528,15.481430,10.778663,7.214802
2,19.067555,8.936035,9.723145,23.556152,8.759773,14.424616,8.595637,4.686904,14.959910,9.191804,...,7.547888,5.305796,3.775112,1.165003,5.901559,2.986440,6.210449,21.278326,10.329825,2.593227
3,3.754010,24.279506,17.814114,34.787143,20.106352,29.657194,21.925127,18.856003,0.587630,6.456432,...,21.790917,20.850563,19.246265,16.614369,21.274591,18.530387,18.221826,32.307155,6.573621,13.021060
4,12.077701,15.570586,10.223952,26.930142,11.752423,20.904164,13.261463,10.085694,8.202609,2.351905,...,13.031531,12.335338,10.911982,8.284289,12.600943,10.084657,9.620927,24.474894,3.265702,4.545431


In [ ]:
candidate_df.shape

(72, 2)

In [ ]:
# Optimization Function Parameter
x = 2    # Number of new ambulances to place

# Risk arrays (length = # of test points)
risk_models = {
    # Kernel Density Estimation model risks
    'KDE': train_risk_kde,
    # Logistic regression risks
    'Logistic': train_risk_logistic,
    # Random forest risks
    'RandomForest': train_risk_rf
}

# Initialize dictionary to store selected coordinates for each model
selected_coords = {}

# Loop through each risk model and optimize ambulance placement
for model_name, risk in risk_models.items():
    # Call the optimization function
    coords = optimize_ambulance_placement(risk, d_current, d_candidate, x, candidate_df)

    # Store the result in the dictionary
    selected_coords[model_name] = coords

# Output the results
print("\n---------------\n")
print("Selected ambulance locations by risk model:")
for model_name, coords in selected_coords.items():
    print(f"{model_name}:\n{coords}\n")

KeyError: 0

# 6. Model Evaluation

## Import Test Dataset

In [ ]:
# Import set of points that weren't used for training the prediction model

# Set viewing permission to 'Anyone with the link' if not already done
# File ID from the Google Drive link -> https://drive.google.com/file/d/FILE_ID/view?usp=sharing
file_id = '1G5Q08o-LKqXiFyhdwAddtjrC6riz0ckC'
url = f'https://drive.google.com/uc?id={file_id}'

# Download the file and save the name of the file
test_dataset_encoded = 'test_dataset_encoded.csv'
gdown.download(url, test_dataset_encoded, quiet=True)

# Loading the .csv into a DataFrame
test_encoded_df = pd.read_csv(test_dataset_encoded)

In [ ]:
# Get test dataset - same random state as what was used for the above
X_train, X_test = train_test_split(df, test_size=0.2, random_state=42)

# Filter to unaltered coordinates (no onehot encoding)
test_coordinates = X_test[['COLLISION_LONGITUDE', 'COLLISION_LATITUDE']].to_numpy()

In [ ]:
#Get size of test_coordinates
test_coordinates.shape

(36724, 2)

## Import Distance CSV (Test)

In [ ]:
# Download KDE
file_id = '1xJpBDTu60ppIr1eB-4FpLJL9FIQa8TS4'
url = f'https://drive.google.com/uc?id={file_id}'
# Download the KDE file and save the name of the file
gdown.download(url, 'kde_current_distance.csv', quiet=False)

# Download Logistic
file_id = '1t7KwEGpZtDt_N_xnbM7ybArdTPfvoNph'
url = f'https://drive.google.com/uc?id={file_id}'
# Download the Logistic file and save the name of the file
gdown.download(url, 'logistic_current_distance.csv', quiet=False)

# Download RF
file_id = '1Dn1UDBphYsevT7Yvo4I1VNehtf0bCP1Z'
url = f'https://drive.google.com/uc?id={file_id}'
# Download the RF file and save the name of the file
gdown.download(url, 'rf_current_distance.csv', quiet=False)

# Download Current
file_id = '1S_XYFtcD9HgjYWRM7qr5trL6WRW5xRnW'
url = f'https://drive.google.com/uc?id={file_id}'
# Download the RF file and save the name of the file
gdown.download(url, 'test_current_distance.csv', quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1xJpBDTu60ppIr1eB-4FpLJL9FIQa8TS4
To: /content/kde_current_distance.csv
100%|██████████| 1.35M/1.35M [00:00<00:00, 50.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1t7KwEGpZtDt_N_xnbM7ybArdTPfvoNph
To: /content/logistic_current_distance.csv
100%|██████████| 1.35M/1.35M [00:00<00:00, 23.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Dn1UDBphYsevT7Yvo4I1VNehtf0bCP1Z
To: /content/rf_current_distance.csv
100%|██████████| 1.35M/1.35M [00:00<00:00, 49.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1S_XYFtcD9HgjYWRM7qr5trL6WRW5xRnW
To: /content/test_current_distance.csv
100%|██████████| 31.2M/31.2M [00:00<00:00, 195MB/s]


'test_current_distance.csv'

##  Compute Distance Matrix

In [ ]:
# Code here (distance between current_df coordinates + "selected_coords" from optimal solution,
# and the number of testing points)

#Distance matrix for current ambulances
test_distance_matrix = pd.read_csv('test_current_distance.csv').to_numpy()

#Distance matrix for KDE optimal ambulances
distance_matrix_kde = pd.read_csv('kde_current_distance.csv').to_numpy()

#Distance matrix for Logistic optimal ambulances
distance_matrix_logistic = pd.read_csv('logistic_current_distance.csv').to_numpy()

#Distance matrix for Random Forest optimal ambulances
distance_matrix_rf = pd.read_csv('rf_current_distance.csv').to_numpy()


In [ ]:
#Full KDE distance matrix (current + KDE optimal concatenated)
#----------REPLACE WITH ACTUAL CODE---------
kde_distance_matrix = np.concatenate((test_distance_matrix, distance_matrix_kde), axis=1)

#Full Logistic distance matrix (current + Logistic optimal concatenated)
logistic_distance_matrix = np.concatenate((test_distance_matrix, distance_matrix_logistic), axis=1)

#Full Random Forest distance matrix (current + Random Forest optimal concatenated)
rf_distance_matrix = np.concatenate((test_distance_matrix, distance_matrix_rf), axis=1)

## Create Function to Calculate Weighted Distance

In [ ]:
#Set function to evaluate model
def evaluate_model(risk, distance_matrix):
    """
    Evaluate the sum of weighted distances for a given risk model.

    Parameters:
    - risk: Array of risk values at each test location (size n_test).
    - distance_matrix: 2D NumPy array of distances (size n_test x n_ambulances),
      where each row corresponds to a test location, and columns correspond to
      all ambulances (current + selected candidate locations).

    Returns:
    - total_weighted_distance: A single float representing the evaluation metric.
    """
    #For each test point, find the distance to the nearest ambulance
    min_distances = np.min(distance_matrix, axis=1)

    # Compute the sum of weighted distances (risk * distance)
    total_weighted_distance = np.sum(risk * min_distances)

    #Return weighted distance
    return total_weighted_distance


## Test Function (No risk)

In [ ]:
#Number of test points
n = test_encoded_df.shape[0]

#Get risk arrays (no risk)
test_risk = np.ones(n)

# Store evaluation results
results = {}

#Get results for KDE
results['KDE'] = evaluate_model(test_risk, kde_distance_matrix)

#Get results for Logistic
results['Logistic'] = evaluate_model(test_risk, logistic_distance_matrix)

#Get results for Random Forest
results['RandomForest'] = evaluate_model(test_risk, rf_distance_matrix)

#Get results for original ambulance locations
results['Original'] = evaluate_model(test_risk, test_distance_matrix)


# Output results
print("\n----------------")
print("Evaluation Results:")
for model_name, score in results.items():
    print(f"{model_name}:")
    print(f"Distance: {score}")

# Find the best model
best_model = min(results, key=results.get)
print("\n----------------")
print(f"Best risk model: {best_model} with distance (score) {results[best_model]}")


----------------
Evaluation Results:
KDE:
Distance: 52690.98700097587
Logistic:
Distance: 52317.798960230655
RandomForest:
Distance: 52317.798960230655
Original:
Distance: 53621.03778733143

----------------
Best risk model: Logistic with distance (score) 52317.798960230655


## Feature Importances

## Logistic Regression

In [ ]:
# Get the coefficients of the features
coefficients = logistic_model.coef_[0]  # For binary classification, it will be a 1D array

# Get the indices of features sorted by the absolute value of coefficients (top features)
top_indices = np.argsort(np.abs(coefficients))[::-1][:20]

# Extract the top 10 feature names using the indices
top_10_feature_names_logreg = test_encoded_df.columns[top_indices]

# Print the top 10 feature names and their coefficients
print("Top 10 Features based on Coefficients (Logistic Regression):")
for i, idx in enumerate(top_indices):
    print(f"Rank {i+1}: Feature '{top_10_feature_names_logreg[i]}', Coefficient: {coefficients[idx]}")

Top 10 Features based on Coefficients (Logistic Regression):
Rank 1: Feature 'OCC_HOUR_0.0', Coefficient: -5.182234272999817
Rank 2: Feature 'OCC_HOUR_15.0', Coefficient: 0.5524721779875705
Rank 3: Feature 'OCC_HOUR_17.0', Coefficient: 0.520427280612705
Rank 4: Feature 'OCC_HOUR_16.0', Coefficient: 0.5201767024470426
Rank 5: Feature 'OCC_HOUR_14.0', Coefficient: 0.512831190702437
Rank 6: Feature 'OCC_HOUR_12.0', Coefficient: 0.5075362350430231
Rank 7: Feature 'OCC_HOUR_13.0', Coefficient: 0.49875938834534894
Rank 8: Feature 'OCC_HOUR_18.0', Coefficient: 0.4835769696252018
Rank 9: Feature 'OCC_HOUR_11.0', Coefficient: 0.45477534169993866
Rank 10: Feature 'NUM_TRAFFIC_SIGNALS_0.0', Coefficient: -0.4228236640891792
Rank 11: Feature 'OCC_HOUR_10.0', Coefficient: 0.4036833407936249
Rank 12: Feature 'OCC_HOUR_8.0', Coefficient: 0.4020335580196308
Rank 13: Feature 'OCC_HOUR_9.0', Coefficient: 0.398767535922144
Rank 14: Feature 'OCC_HOUR_19.0', Coefficient: 0.3870802830413537
Rank 15: Feature 

### Random Forest

In [ ]:
# Get feature importances
importances = rf_model.feature_importances_

# Get the indices of the top 10 features
top_10_indices = np.argsort(importances)[::-1][:10]

# Get the top 10 importances and their corresponding indices
top_10_importances = importances[top_10_indices]

# Print top 10 feature importances with indices
print("Top 10 Feature Importances and their Indices:")
for i, idx in enumerate(top_10_indices):
    print(f"Rank {i+1}: Feature index {idx}, Importance: {top_10_importances[i]}")

Top 10 Feature Importances and their Indices:
Rank 1: Feature index 102, Importance: 0.15626589170580857
Rank 2: Feature index 103, Importance: 0.15597150740063623
Rank 3: Feature index 104, Importance: 0.07897942338033359
Rank 4: Feature index 108, Importance: 0.07870368210315241
Rank 5: Feature index 105, Importance: 0.07179040382095211
Rank 6: Feature index 107, Importance: 0.012593366437470242
Rank 7: Feature index 106, Importance: 0.011934498299470228
Rank 8: Feature index 99, Importance: 0.00936800242290564
Rank 9: Feature index 92, Importance: 0.008850125330329887
Rank 10: Feature index 94, Importance: 0.008763960017219005


In [ ]:
# Extract the top 10 feature names using the indices
print("Top 10 Feature Names:")
top_10_feature_names = test_encoded_df.columns[top_10_indices]
top_10_feature_names

Top 10 Feature Names:


Index(['COLLISION_LONGITUDE', 'COLLISION_LATITUDE', 'Temp (°C)',
       'Stn Press (kPa)', 'Rel Hum (%)', 'Visibility (km)',
       'Precip. Amount (mm)', 'NUM_TRAFFIC_SIGNALS_1.0', 'SPD_KM_40',
       'SPD_KM_60'],
      dtype='object')